In [20]:
pip install streamlit

In [ ]:
!git clone https://github.com/ultralytics/ultralytics
!pip install ultralytics

fatal: destination path 'ultralytics' already exists and is not an empty directory.


In [50]:
# input sample asset

import gdown
import
download_link = 'https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z'
output_path = '/content/video-contoh-trim.mov'

gdown.download(download_link, output_path, quiet=False)
print('File berhasil diunduh dan disimpan di:', output_path)


Downloading...
From (original): https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z
From (redirected): https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z&confirm=t&uuid=989a394e-796c-4770-ab49-7760ec3180b5
To: /content/video-contoh-trim.mov
100%|██████████| 206M/206M [00:01<00:00, 165MB/s]

File berhasil diunduh dan disimpan di: /content/video-contoh-trim.mov


In [64]:
%%writefile settings.py
from pathlib import Path
import sys

file_path = Path(__file__).resolve()
root_path = file_path.parent

if root_path not in sys.path:
    sys.path.append(str(root_path))

ROOT = root_path.relative_to(Path.cwd())

# Sources
IMAGE = 'Image'
VIDEO = 'Video'

SOURCES_LIST = [IMAGE, VIDEO]

IMAGES_DIR = ROOT / 'images'

VIDEO_DIR = Path('/content')
VIDEO_1_PATH = VIDEO_DIR / 'video-contoh-trim.mov'

VIDEOS_DICT = {
    'video_1': VIDEO_1_PATH,

}

# Model config
MODEL_DIR = ROOT / 'weights'
DETECTION_MODEL = MODEL_DIR / 'yolov8n.pt'
# DETECTION_MODEL = MODEL_DIR / 'my_detection_model.pt'

SEGMENTATION_MODEL = MODEL_DIR / 'yolov8n-seg.pt'

Overwriting settings.py


In [72]:
%%writefile helper.py

from ultralytics import YOLO
import time
import streamlit as st
import cv2
import settings

def load_model(model_path):
    model = YOLO(model_path)
    return model


def display_tracker_options():
    display_tracker = st.radio("Display Tracker", ('Yes', 'No'))
    is_display_tracker = True if display_tracker == 'Yes' else False
    if is_display_tracker:
        tracker_type = st.radio("Tracker", ("bytetrack.yaml", "botsort.yaml"))
        return is_display_tracker, tracker_type
    return is_display_tracker, None


def _display_detected_frames(conf, model, st_frame, image, is_display_tracking=None, tracker=None):


    # Resize the image to a standard size
    image = cv2.resize(image, (720, int(720*(9/16))))

    # Display object tracking, if specified
    if is_display_tracking:
        res = model.track(image, conf=conf, persist=True, tracker=tracker)
    else:
        # Predict the objects in the image using the YOLOv8 model
        res = model.predict(image, conf=conf)

    # # Plot the detected objects on the video frame
    res_plotted = res[0].plot()
    st_frame.image(res_plotted,
                   caption='Detected Video',
                   channels="BGR",
                   use_column_width=True
                   )


def play_stored_video(conf, model):
    source_vid = st.sidebar.selectbox(
        "Choose a video...", settings.VIDEOS_DICT.keys())

    is_display_tracker, tracker = display_tracker_options()

    with open(settings.VIDEOS_DICT.get(source_vid), 'rb') as video_file:
        video_bytes = video_file.read()
    if video_bytes:
        st.video(video_bytes)

    if st.sidebar.button('Detect Video Objects'):
        try:
            vid_cap = cv2.VideoCapture(
                str(settings.VIDEOS_DICT.get(source_vid)))
            st_frame = st.empty()
            while (vid_cap.isOpened()):
                success, image = vid_cap.read()
                if success:
                    _display_detected_frames(conf,
                                             model,
                                             st_frame,
                                             image,
                                             is_display_tracker,
                                             tracker
                                             )
                else:
                    vid_cap.release()
                    break
        except Exception as e:
            st.sidebar.error("Error loading video: " + str(e))

Overwriting helper.py


In [70]:
%%writefile app.py
import streamlit as st

from pathlib import Path
import PIL

# Local Modules
import settings
import helper
import asset

st.set_page_config(
    page_title="Consctruction Safety Detection using YOLO",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Main page
st.header("Consctruction Safety Detection using YOLO", divider='rainbow')

# Sidebar
st.sidebar.header("Configuration")
confidence = float(st.sidebar.slider( "Select Model Confidence", 25, 100, 40)) / 100
model_path = Path(settings.DETECTION_MODEL)

# Load Pre-trained Model
try:
    model = helper.load_model(model_path)
except Exception as ex:
    st.error(f"Unable to load model. Check path: {model_path}")
    st.error(ex)

st.sidebar.header("Image/Video Config")
source_radio = st.sidebar.radio(
    "Select Source", settings.SOURCES_LIST)

source_img = None

# If image is selected
if source_radio == settings.IMAGE:
    source_img = st.sidebar.file_uploader(
        "Choose an image...", type=("jpg", "jpeg", "png", 'bmp', 'webp'))

    col1, col2 = st.columns(2)

    with col1:
        try:
            if source_img is None:
                st.image(st.image("https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon@2.png", width=400), caption="Default Image", use_column_width=True)
            else:
                uploaded_image = PIL.Image.open(source_img)
                st.image(source_img, caption="Uploaded Image",
                         use_column_width=True)
        except Exception as ex:
            st.error(ex)

    with col2:
        if source_img is None:
            st.subheader('Masukkan gambar atau video anda terlebih dahulu')
        else:
            if st.sidebar.button('Detect Objects'):
                res = model.predict(uploaded_image,
                                    conf=confidence
                                    )
                boxes = res[0].boxes
                res_plotted = res[0].plot()[:, :, ::-1]
                st.image(res_plotted, caption='Detected Image',
                         use_column_width=True)
                try:
                    with st.expander("Detection Results"):
                        for box in boxes:
                            st.write(box.data)
                except Exception as ex:
                    st.write("No image is uploaded yet!")

elif source_radio == settings.VIDEO:
  helper.play_stored_video(confidence, model)

else:
    st.error("Please select a valid source type!")

Overwriting app.py


In [73]:
!streamlit run app.py & npx localtunnel --port 8501



npx: installed 22 in 4.951s
your url is: https://five-islands-appear.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.30.221:8501

Downloading...
From (original): https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z
From (redirected): https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z&confirm=t&uuid=0a7c202d-e65d-48ca-b9f8-00b2e1eaafad
To: /content/video-contoh-trim.mov
100% 206M/206M [00:01<00:00, 141MB/s]
File berhasil diunduh dan disimpan di: /content/video-contoh-trim.mov

0: 480x640 3 persons, 245.9ms
Speed: 8.8ms preprocess, 245.9ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 1 truck, 211.8ms
Speed: 3.6ms preprocess, 211.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 146.3ms
Speed: 3.7ms preprocess, 146.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 14